## Nosso dataset :

In [ ]:
!wget https://github.com/otaviofcoletti/Sara_dataset/raw/main/Sara_dataset_direito.zip

In [1]:
!unzip Sara_dataset_direito.zip


Archive:  Sara_dataset_direito.zip
replace Sara_dataset_direito/Situação rolo 3 da esquerda levantado index 14.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [5]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from sklearn.datasets import make_blobs
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from aeon.classification.convolution_based import RocketClassifier
from aeon.classification.convolution_based import Arsenal, RocketClassifier

from numpy import mean
from numpy import std
from sklearn.datasets import make_blobs
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

import pandas as pd
import plotly.express as px
from sklearn.metrics import confusion_matrix

from aeon.classification.hybrid import HIVECOTEV2
from aeon.classification.feature_based import Catch22Classifier
from aeon.transformations.collection.catch22 import Catch22


In [2]:
lista_de_arquivos = os.listdir("Sara_dataset_direito")

In [3]:
# Function to determine the unit based on the sensor type
def get_unit(sensor):
    if sensor == "Gyroscope":
        unit = "(deg/s)"
    else:
        unit = "(g)"
    return unit

In [4]:
label_map = {
        "Situação referencia index": 0,
        "Situação rolo 3 da esquerda levantado index": 1,
        "Situação rolo 3 da esquerda removido index": 2,
        "Situação rolo 3 da direita levantado index": 3,
        "Situação rolo 3 da direita removido index": 4,
    }



# Random Forest (dados brutos) 5 classes


In [6]:


# Assuming 'label_map' and 'lista_de_arquivos' are defined

casos = []
labels = []

for i in lista_de_arquivos:
    for j in label_map:
        if j in i:
            labels.append(label_map[j])
            df = pd.read_csv(f"Sara_dataset_direito/{i}")
            
            # Combine Gyroscope and Accelerometer features
            features = df[['Gyroscope X (deg/s)', 'Gyroscope Y (deg/s)', 'Gyroscope Z (deg/s)',
                            'Accelerometer X (g)', 'Accelerometer Y (g)', 'Accelerometer Z (g)']]
            
            # You can add more feature engineering or preprocessing steps here
            
            casos.append(features)

X = np.array(casos)
y = np.array(labels)

# Flatten or reshape the array to make it two-dimensional
X = np.array(casos).reshape(len(casos), -1)

# Random Forest CV

In [7]:
# loocv to automatically evaluate the performance of a random forest classifier
cv = LeaveOneOut()
# create model
rf = RandomForestClassifier()

# evaluate model
pred = cross_val_predict(rf, X, y, cv=cv, n_jobs=-1)
# report performance


In [8]:
accuracy = accuracy_score(y, pred)
print("Accuracy:", accuracy)

Accuracy: 0.8269230769230769


In [9]:

# Convert lists to numpy arrays
y = np.array(y)
pred = np.array(pred)

# Binarize the labels
y_binarized = label_binarize(y, classes=np.unique(y))

# Ensure pred is a 2D array with one column per class
pred = pred.reshape(-1, 1) if pred.ndim == 1 else pred

# Binarize the predicted labels
pred_binarized = label_binarize(pred, classes=np.unique(y))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

ROC AUC Score: 0.8894373822224078


In [13]:
conf_matrix = confusion_matrix(y, pred)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_image('random_forest.pdf')


# Mini Rocket

In [14]:

cv = LeaveOneOut()
# create model
multi_r = Arsenal(rocket_transform="minirocket")

# evaluate model
pred = cross_val_predict(multi_r, np.array(X), np.array(y), cv=cv, n_jobs=-1)
# report performance

/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/otavio/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The T

In [15]:
accuracy = accuracy_score(y, pred)
print("Accuracy:", accuracy)

Accuracy: 0.9038461538461539


In [16]:

# Convert lists to numpy arrays
y = np.array(y)
pred = np.array(pred)

# Binarize the labels
y_binarized = label_binarize(y, classes=np.unique(y))

# Ensure pred is a 2D array with one column per class
pred = pred.reshape(-1, 1) if pred.ndim == 1 else pred

# Binarize the predicted labels
pred_binarized = label_binarize(pred, classes=np.unique(y))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

ROC AUC Score: 0.9388660475047317


In [20]:
conf_matrix = confusion_matrix(y, pred)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=20, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_image('minirocket.pdf')


# Catch22 + Isolation Forest 2 classes

In [46]:
# loocv to automatically evaluate the performance of a random forest classifier

cv = LeaveOneOut()
# create model
clf = Catch22Classifier()

# evaluate model
pred = cross_val_predict(clf, np.array(X), np.array(y), cv=cv, n_jobs=-1)
# report performance


KeyboardInterrupt: 

In [ ]:
accuracy = accuracy_score(y, pred)
print("Accuracy:", accuracy)

In [ ]:
y = np.array(y)
pred = np.array(pred)

# Binarize the labels
y_binarized = label_binarize(y, classes=np.unique(y))

# Ensure pred is a 2D array with one column per class
pred = pred.reshape(-1, 1) if pred.ndim == 1 else pred

# Binarize the predicted labels
pred_binarized = label_binarize(pred, classes=np.unique(y))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

In [ ]:
conf_matrix = confusion_matrix(y, pred)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_html('catch22.html')


# HIVECOTE

In [ ]:

cv = LeaveOneOut()
# create model
hc2 = HIVECOTEV2(time_limit_in_minutes=0.2)

# evaluate model
pred = cross_val_predict(hc2, np.array(X), np.array(y), cv=cv, n_jobs=-1)
# report performance


In [ ]:
accuracy = accuracy_score(y, pred)
print("Accuracy:", accuracy)

In [ ]:
y = np.array(y)
pred = np.array(pred)

# Binarize the labels
y_binarized = label_binarize(y, classes=np.unique(y))

# Ensure pred is a 2D array with one column per class
pred = pred.reshape(-1, 1) if pred.ndim == 1 else pred

# Binarize the predicted labels
pred_binarized = label_binarize(pred, classes=np.unique(y))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

In [ ]:
conf_matrix = confusion_matrix(y, pred)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_html('hivecote2.html')


# Time Series Forest

In [ ]:
from pyts.datasets import load_gunpoint
from pyts.classification import TimeSeriesForest

X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.25, random_state=42)
clf = TimeSeriesForest(random_state=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_test, y_test)

In [ ]:
cv = LeaveOneOut()
# create model
clf = TimeSeriesForest()

# evaluate model
pred = cross_val_predict(clf, np.array(X), np.array(y), cv=cv, n_jobs=-1)
# report performance

In [ ]:
accuracy = accuracy_score(y, pred)
print("Accuracy:", accuracy)

In [ ]:
y = np.array(y)
pred = np.array(pred)

# Binarize the labels
y_binarized = label_binarize(y, classes=np.unique(y))

# Ensure pred is a 2D array with one column per class
pred = pred.reshape(-1, 1) if pred.ndim == 1 else pred

# Binarize the predicted labels
pred_binarized = label_binarize(pred, classes=np.unique(y))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

In [ ]:
conf_matrix = confusion_matrix(y, pred)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_html('ts_forest.html')


# Rocket Multirocket 

In [ ]:
cv = LeaveOneOut()
# create model
multi_r = Arsenal(rocket_transform="multirocket")

# evaluate model
pred = cross_val_predict(multi_r, np.array(X), np.array(y), cv=cv, n_jobs=-1)
# report performance

In [ ]:
accuracy = accuracy_score(y, pred)
print("Accuracy:", accuracy)

In [ ]:
y = np.array(y)
pred = np.array(pred)

# Binarize the labels
y_binarized = label_binarize(y, classes=np.unique(y))

# Ensure pred is a 2D array with one column per class
pred = pred.reshape(-1, 1) if pred.ndim == 1 else pred

# Binarize the predicted labels
pred_binarized = label_binarize(pred, classes=np.unique(y))

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_binarized, pred_binarized, multi_class='ovr')

print(f'ROC AUC Score: {roc_auc}')

In [ ]:
conf_matrix = confusion_matrix(y, pred)

class_names = ['Right Up', 'Right Down', 'Left Un.', 'Normal', 'Left Up']


# Create a DataFrame from the confusion matrix
confusion_df = pd.DataFrame(conf_matrix,
                             columns=[f'{class_names[i]}' for i in range(conf_matrix.shape[1])],
                             index=[f'{class_names[i]}' for i in range(conf_matrix.shape[0])])


# Plot the heatmap
heatmap = px.imshow(confusion_df, labels=dict(x="Predicted", y="Actual", color="Count"), text_auto=True)

heatmap.update_layout(title='Confusion Matrix',
                      xaxis_nticks=5, yaxis_nticks=5,
                      font=dict(size=14, family='Courier New, monospace'),
                      width=800, height=800)

heatmap.show()

heatmap.write_html('multirocket.html')
